# Pre Process Dataset

In [ ]:
import os

import glob
import logging
import multiprocessing as mp
import time
import hashlib

import cv2
import mtcnn
import numpy as np

from skimage.transform import resize

In [ ]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

In [ ]:
def main(input_dir, output_dir, align=False, crop_dim=(160,160), margin=32):
    """
    param: input_dir: Contains other directories with images named by identity
    param: output_dir: Path where faces are stored after being proccesed
    param: crop_dim: (H,W) dimensions
    """
    start_time = time.time()
    #pool = mp.Pool(processes=mp.cpu_count())

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for image_dir in os.listdir(input_dir):
        image_output_dir = os.path.join(output_dir, os.path.basename(image_dir))
        if not os.path.exists(image_output_dir):
            os.makedirs(image_output_dir)

    image_paths = glob.glob(os.path.join(input_dir, '**/*.jpg'))
    for image_path in image_paths:
        logger.info('Processing: {}'.format(image_path))
        image_output_dir = os.path.join(output_dir, os.path.basename(os.path.dirname(image_path)))
        #pool.apply_async(preprocess_image, (image_path, image_output_dir, crop_dim))
        preprocess_image(image_path, image_output_dir, align, crop_dim, margin)

    #pool.close()
    #pool.join()
    #logger.info('Completed in {} seconds'.format(time.time() - start_time))
    print('Completed in {} seconds'.format(time.time() - start_time))

In [ ]:
def preprocess_image(image_path, image_output_dir, align, crop_dim, margin):
    """
    param: image_path: The path of the image
    param: image_output_dir: The directory where the output faces are stored
    param: crop_dim:
    param: margin: Add a margin to the detected coordinates of the face 
    """
    # Remove extension from file name keeping the image ID
    identity = os.path.basename(image_path).split('.')[0]
    output_path = os.path.join(image_output_dir, identity)
    
    logger.info('Reading image: {}'.format(image_path))
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Skip processing if image is already there
    if len(glob.glob('{}*.jpg'.format(output_path))) < 1:
        faces = detect_faces(image, margin)
        logger.info('Faces: {}'.format(len(faces)))

        for index, face in enumerate(faces):
            if align:
                image = align_face(image, face['keypoints'])
            cropped = image[face['right']:face['top'], face['left']:face['bottom'], :]
            cropped = cv2.resize(cropped, crop_dim, interpolation = cv2.INTER_AREA)
            cropped = cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB)  
            # Store cropped
            cv2.imwrite('{}-{}.jpg'.format(output_path, index), cropped)

In [ ]:
def detect_faces(image, margin):
    """
    Return a list of dictionaries with coordinates locating faces in the image.
    """
    coordinates = []
    # Create the detector, using default weights
    detector = mtcnn.MTCNN()
    faces = detector.detect_faces(image)
    
    if len(faces) == 1:
        confidence_threshold = 0.80
    elif len(faces) > 1:
        confidence_threshold = 0.999
            
    for index, face in enumerate(faces):
        logger.info('Face confidence: {}'.format(face['confidence']))

        if face['confidence'] >= confidence_threshold:
            (x, y, w, h) = face['box']
            
            img_size = np.asarray(image.shape)[0:2]
            bb = {}
            bb['left'] = max(x-margin//2, 0)
            bb['right'] = max(y-margin//2, 0)
            bb['bottom'] = min(x+w+margin//2, img_size[1])
            bb['top'] = min(y+h+margin//2, img_size[0])
            bb['keypoints'] = face['keypoints']
            
            coordinates.append(bb)
            
    return coordinates

In [ ]:
def align_face(img, keypoints):
    """
    Return the face aligned based on keypoints
    
    Reference:
    https://github.com/jrosebr1/imutils/blob/master/imutils/face_utils/facealigner.py
    """
    (h, w) = img.shape[:2]
        
    # compute the angle between the eye centroids
    dy = keypoints['right_eye'][1] - keypoints['left_eye'][1]
    dx = keypoints['right_eye'][0] - keypoints['left_eye'][0]
    angle = -np.degrees(np.arctan2(dy, dx))
    logger.info('Angle: ({}, {}): {}'.format(dx, dy, angle))
    print('Angle: ({}, {}): {}'.format(dx, dy, angle))

    # compute center (x, y)-coordinates (i.e., the median point)
    # between the two eyes in the input image
    eyes_center = ((keypoints['left_eye'][0] + keypoints['right_eye'][0]) // 2,
                  (keypoints['left_eye'][1] + keypoints['right_eye'][1]) // 2)

    # grab the rotation matrix for rotating and scaling the face
    M = cv2.getRotationMatrix2D(eyes_center, angle, scale=1.0)
    aligned = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_CUBIC)
        
    return aligned

In [ ]:
def validate_dataset(dataset_dir):
    hashes = {}
    
    image_paths = glob.glob(os.path.join(dataset_dir, '**/*.jpg'))
    for image_path in image_paths:
        image = cv2.imread(image_path)
        h = hashlib.sha256(image).hexdigest()
        if h in hashes.keys():
            print('Image is already indexed ({})'.format(h))
            print(hashes[h])
            print(image_path)
        else:
            hashes[h] = image_path

## Process images

In [ ]:
input_dir = '../data/images/'
output_dir = '../data/images-out/'

main(input_dir, output_dir, align=False)

In [ ]:
validate_dataset(output_dir)